# 23-24 测试的基本规则和流程
我们已经学习完涉及Go语言所有的内建数据类型，以及特色的流程和语句。
理论上说我们已经完全可以独立编写各种各样的go程序了，如果有遗忘回到之前的文章复习一下就行。

接下来我们主要学习go语言编程进阶，掌握附加知识比如：Go程序测试，程序监测，以及Go语言标准库中各种常用代码包的正确使用方法。

对于程序和软件来讲，尽早发现问题以及修正问题其实非常重要。对于优秀的程序和软件来说，测试必然是非常受重视的一个环节，所以我们最好利用测试来为程序建起堡垒。

对于程序或软件的测试也分很多种，比如：单元测试、API测试、集成测试、灰度测试，等等。我在本模块会主要针对单元测试进行讲解。

# 前导内容：go程序测试基础知识
## 单元测试
单元测试又称程序员测试，顾名思义这就是程序员们本来应该做的自我检查工作之一。
Go语言的创造者们非常重视程序测试，为Go程序开发者们提供了丰富的API和工具。利用这些工具，我们可以创建测试源码文件，并为明亮源码文件和库源码文件中的程序实体编写测试用例。

在Go语言中，一个测试用例往往会由一个或多个测试函数来代表，不过在大多数情况下，每个测试用例仅用一个测试函数就足够了。测试函数往往用于描述和保障某个程序实体的某方面功能，比如，该功能在正常情况下会因什么样的输入，产生什么样的输出，又比如，该功能会在什么情况下报错或表现异常，等等。

我们可以为Go程序编写三类测试，即：功能测试（test）、基准测试（benchmark，也称性能测试），以及示例测试（example）。

对于前两类测试，从名称上你就应该可以猜到它们的用途。而示例测试严格来讲也是一种功能测试，只不过它更关注程序打印出来的内容。

一般情况下，一个测试源码文件只会针对于某个命令源码文件，或库源码文件（以下简称被测源码文件）做测试，所以我们总会（并且应该）把它们放在同一个代码包内。

测试源码文件的主名称应该以被测源码文件的主名称为前导，并且必须以“_test”为后缀。例如，如果被测源码文件的名称为demo52.go，那么针对它的测试源码文件的名称就应该是demo52_test.go。

每个测试源码文件都必须至少包含一个测试函数。并且，从语法上讲，每个测试源码文件中，都可以包含用来做任何一类测试的测试函数，即使把这三类测试函数都塞进去也没有问题。我通常就是这么做的，只要把控好测试函数的分组和数量就可以了。

我们可以依据这些测试函数针对的不同程序实体，把它们分成不同的**逻辑组**，并且，利用注释以及帮助类的变量或函数来做分割。同时，我们还可以依据被测源码文件中程序实体的先后顺序，来安排测试源码文件中测试函数的顺序。

此外，不仅仅对测试源码文件的名称，对于测试函数的名称和签名，Go语言也是有明文规定的。你知道这个规定的内容吗？---不知道:)

所以，我们今天的问题就是：Go语言对测试函数的名称和签名都有哪些规定？

这里我给出的典型回答是下面三个内容。

- 对于功能测试函数来说，其名称必须以Test为前缀，并且参数列表中只应有一个*testing.T类型的参数声明。
- 对于性能测试函数来说，其名称必须以Benchmark为前缀，并且唯一参数的类型必须是*testing.B类型的。
- 对于示例测试函数来说，其名称必须以Example为前缀，但对函数的参数列表没有强制规定。

# 问题解析：
我问这个问题的目的一般有两个。

- 第一个目的当然是考察Go程序测试的基本规则。如果你经常编写测试源码文件，那么这道题应该是很容易回答的。

- 第二个目的是作为一个引子，引出第二个问题，即：go test命令执行的主要测试流程是什么？不过在这里我就不问你了，我直接说一下答案。

我们首先需要记住一点，只有测试源码文件的名称对了，测试函数的名称和签名也对了，当我们运行`go test`命令的时候，其中的测试代码才有可能被运行。

`go test`命令在开始运行时，会先做一些准备工作，比如，确定内部需要用到的命令，检查我们指定的代码包或源码文件的有效性，以及判断我们给予的标记是否合法，等等。

在准备工作顺利完成之后，`go test`命令就会针对每个被测代码包，依次地进行构建、执行包中符合要求的测试函数，清理临时文件，打印测试结果。这就是通常情况下的主要测试流程。

请注意上述的“依次”二字。对于每个被测代码包，`go test`命令会串行地执行测试流程中的每个步骤。

但是，为了加快测试速度，它通常会并发地对多个被测代码包进行功能测试，只不过，在最后打印测试结果的时候，它会依照我们给定的顺序逐个进行，这会让我们感觉到它是在完全串行地执行测试流程。

另一方面，由于并发的测试会让性能测试的结果存在偏差，所以性能测试一般都是串行进行的。更具体地说，只有在所有构建步骤都做完之后，`go test`命令才会真正地开始进行性能测试。

并且，下一个代码包性能测试的进行，总会等到上一个代码包性能测试的结果打印完成才会开始，而且性能测试函数的执行也都会是串行的。

一旦清楚了Go程序测试的具体过程，我们的一些疑惑就自然有了答案。比如，那个名叫`testIntroduce`的测试函数为什么没执行，又比如，为什么即使是简单的性能测试执行起来也会比功能测试慢，等等。

In [1]:
package main

import (
	"errors"
	"flag"
	"fmt"
)

var name string

func init() {
	flag.StringVar(&name, "name", "everyone", "The greeting object.")
}

func main() {
	flag.Parse()
	greeting, err := hello(name)
	if err != nil {
		fmt.Printf("error: %s\n", err)
		return
	}
	fmt.Println(greeting, introduce())
}

// hello 用于生成问候内容。
func hello(name string) (string, error) {
	if name == "" {
		return "", errors.New("empty name")
	}
	return fmt.Sprintf("Hello, %s!", name), nil
}

// introduce 用于生成介绍内容。
func introduce() string {
	return "Welcome to my Golang column."
}


Hello, everyone! Welcome to my Golang column.


In [ ]:
package main

import (
	"fmt"
	"testing"
)

func TestHello(t *testing.T) {
	//测试用例1：name为空字符串
	var name string
	greeting, err := hello(name)
	if err == nil {
		t.Errorf("The error is nil, but it should not be. (name=%q)",
			name)
	}
	if greeting != "" {
		t.Errorf("Nonempty greeting, but it should not be. (name=%q)",
			name)
	}
	//测试用例2：name为非空字符串
	name = "Robert"
	greeting, err = hello(name)
	if err != nil {
		t.Errorf("The error is not nil, but it should be. (name=%q)",
			name)
	}
	if greeting == "" {
		t.Errorf("Empty greeting, but it should not be. (name=%q)",
			name)
	}
	expected := fmt.Sprintf("Hello, %s!", name)
	if greeting != expected {
		t.Errorf("The actual greeting %q is not the expected. (name=%q)",
			greeting, name)
	}
	t.Logf("The expected greeting is %q.\n", expected)
}

func TestIntroduce(t *testing.T) { // 请注意这个测试函数的名称。
	intro := introduce()
	expected := "Welcome to my Golang column."
	if intro != expected {
		t.Errorf("The actual introduce %q is not the expected.",
			intro)
	}
	t.Logf("The expected introduce is %q.\n", expected)
}


# 总结
在本篇文章的一开始，我就试图向你阐释程序测试的重要性。在我经历的公司中起码有一半都不重视程序测试，或者说没有精力去做程序测试。

尤其是中小型的公司，他们往往完全依靠软件质量保障团队，甚至真正的用户去帮他们测试。在这些情况下，软件错误或缺陷的发现、反馈和修复的周期通常会很长，成本也会很大，也许还会造成很不好的影响。

Go语言是一门很重视程序测试的编程语言，它不但自带了testing包，还有专用于程序测试的命令go test。我们要想真正用好一个工具，就需要先了解它的核心逻辑。所以，我今天问你的第一个问题就是关于go test命令的基本规则和主要流程的。在知道这些之后，也许你对Go程序测试就会进入更深层次的了解。

# 思考题
除了本文中提到的，你还知道或用过testing.T类型和testing.B类型的哪些方法？它们都是做什么用的？你可以给我留言，我们一起讨论。